## Đọc dataframe và xóa bỏ các trùng lặp


In [1]:
import pandas as pd
import json
from module.module import chuyen_chieu_ngang_sang_m,chuyen_dien_tich_sang_m2


In [2]:
# Đọc file JSON vào DataFrame
df = pd.read_json('../data/data.json')
# Xóa các dòng trùng lặp
df = df.drop_duplicates()

# Đánh số thứ tự lại (reset index)
df = df.reset_index(drop=True)




### Kiểm tra xem có cột nào bị rỗng không

In [3]:
na_full_cols = df.columns[df.isnull().all()]
print("Các cột toàn bộ là NaN:", na_full_cols.tolist())


Các cột toàn bộ là NaN: []


In [4]:
print(df.shape)


(16991, 14)


In [5]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16991 entries, 0 to 16990
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   DiaChi           16990 non-null  object 
 1   LoaiHinhDat      16990 non-null  object 
 2   DienTichDat      16990 non-null  object 
 3   Gia/M2           16971 non-null  object 
 4   GiayToPhapLy     16975 non-null  object 
 5   ChieuNgang       14240 non-null  object 
 6   ChieuDai         13152 non-null  object 
 7   DonVi(M2/Hecta)  10117 non-null  object 
 8   HuongDat         6206 non-null   object 
 9   DacDiemNha/Dat   13352 non-null  object 
 10  TenPhanKhu       1297 non-null   object 
 11  MaLo             189 non-null    object 
 12  Gia/Hecta        19 non-null     object 
 13  TenDuong         4 non-null      float64
dtypes: float64(1), object(13)
memory usage: 1.8+ MB
None


Sau khi kiểm tra thì thấy mặc dù chỉ có 16990 dòng mà các cột TenPhanKhu, MaLo, Gia/Hecta, TenDuong gần như 90% đến ngưỡng 100% là giá trị null nên sẽ drop các cột này

In [6]:
df = df.drop(columns=['TenPhanKhu', 'MaLo', 'Gia/Hecta', 'TenDuong'])


In [7]:
df.head(5)

,DiaChi,LoaiHinhDat,DienTichDat,Gia/M2,GiayToPhapLy,ChieuNgang,ChieuDai,DonVi(M2/Hecta),HuongDat,DacDiemNha/Dat
0,"Đường Hoàng Bá Huân, Thị trấn Củ Chi, Huyện Củ...",Đất thổ cư,100 m²,"6,50 triệu/m²",Đã có sổ,5 m,20 m,m²,NaN,NaN
1,"Mặt Tiền Sông Vàm Thuật, Phường 6, Quận Gò Vấp...",Đất thổ cư,171 m²,"178,36 triệu/m²",Đã có sổ,9 m,19 m,NaN,NaN,NaN
2,"Đường Mai Đăng Chơn, Phường Hoà Quý, Quận Ngũ ...",Đất thổ cư,90 m²,"21,44 triệu/m²",Đã có sổ,5 m,18 m,NaN,Đông Bắc,Hẻm xe hơi\nMặt tiền\nThổ cư toàn bộ
3,"Gần Làng Đại Học Hồ Chí Minh, Phường Bình An, ...",Đất thổ cư,123 m²,"22,76 triệu/m²",Đã có sổ,4.1 m,NaN,m²,NaN,NaN
4,"DT517, Xã Đồng Lợi, Huyện Triệu Sơn, Thanh Hóa",Đất thổ cư,109 m²,"10,55 triệu/m²",Đã có sổ,5.5 m,20 m,NaN,Đông Nam,Mặt tiền\nNở hậu\nThổ cư toàn bộ


# Chuẩn hóa dữ liệu

## Chuẩn hóa các cột số trước

### Diện Tích Đất

In [8]:
#1 hecta = 10000 mét vuông
# Trích đơn vị bằng cách tách sau dấu cách cuối cùng
df['DonVi_DienTich'] = df['DienTichDat'].astype(str).apply(lambda x: x.split()[-1] if ' ' in x else 'Không rõ')

# Xem tất cả đơn vị xuất hiện
print(df['DonVi_DienTich'].value_counts())
df_khong_ro = df[df['DonVi_DienTich'] == 'Không rõ']
df_hecta = df[df['DonVi_DienTich'] == 'hecta']

print("Các dòng có đơn vị không rõ trong 'DienTichDat':")
print(df_khong_ro[['DienTichDat']])

print("Các dòng có đơn vị không rõ trong 'DienTichDat':")
print(df_hecta[['DienTichDat']])




DonVi_DienTich
m²          16971
hecta          19
Không rõ        1
Name: count, dtype: int64
Các dòng có đơn vị không rõ trong 'DienTichDat':
   DienTichDat
37         NaN
Các dòng có đơn vị không rõ trong 'DienTichDat':
        DienTichDat
331       1.5 hecta
460         4 hecta
1251        9 hecta
2879      9.8 hecta
3180        1 hecta
3807      4.2 hecta
5508   100000 hecta
6162    23000 hecta
6992       60 hecta
7002        5 hecta
7337      2.8 hecta
7773        7 hecta
9570      4.7 hecta
9614        1 hecta
10272     2.4 hecta
10629     6.5 hecta
11003     4.1 hecta
12218     1.4 hecta
15356    2.34 hecta


Có thể thấy là trong cột Diện Tích Đất thì chỉ có 2 dữ liệu là m² và hecta, còn Không rõ là cột bị Nan nên sẽ drop cột đó luôn

In [9]:
df = df.dropna(subset=['DienTichDat'])
print(f"Số dòng còn lại sau khi xoá: {len(df)}")
print(df['DienTichDat'].isnull().sum())  # Kiểm tra còn NaN không


Số dòng còn lại sau khi xoá: 16990
0


In [10]:
df['DienTichDat'] = df['DienTichDat'].apply(chuyen_dien_tich_sang_m2)


### Chiều ngang

In [12]:
df['ChieuNgang'] = df['ChieuNgang'].astype(str).apply(lambda x: x.split()[-1] if ' ' in x else 'Không rõ')

# Xem tất cả đơn vị xuất hiện
print(df['ChieuNgang'].value_counts())
df_khong_ro = df[df['ChieuNgang'] == 'Không rõ']
print("Các dòng có đơn vị không rõ trong 'ChieuNgang':")
print(df_khong_ro[['ChieuNgang']])


ChieuNgang
m           14240
Không rõ     2750
Name: count, dtype: int64
Các dòng có đơn vị không rõ trong 'ChieuNgang':
      ChieuNgang
12      Không rõ
36      Không rõ
42      Không rõ
47      Không rõ
49      Không rõ
...          ...
16971   Không rõ
16980   Không rõ
16981   Không rõ
16982   Không rõ
16990   Không rõ

[2750 rows x 1 columns]
